# Questionnaire Processing

This notebook processes the cleaned questionnaire data, i.e., computes questionnaire sum scores from raw items.

In [ ]:
import json
from pathlib import Path

import biopsykit as bp

from fau_colors import cmaps
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

## Setup Paths

In [ ]:
deploy_type = "local"

In [ ]:
config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))
base_path = Path(config_dict[deploy_type]["base_path"])

quest_path = base_path.joinpath("questionnaire_total")

In [ ]:
output_path = quest_path.joinpath("processed")
bp.utils.file_handling.mkdirs(output_path)

## Load Questionnaire Data

In [ ]:
quest_data = bp.io.load_questionnaire_data(quest_path.joinpath("cleaned/stresspose_questionnaire_cleaned.xlsx"))
quest_data.head()

## Compute BMI

In [ ]:
quest_data = quest_data.assign(**{"BMI": bp.metadata.bmi(quest_data, columns=["weight", "height"])})
quest_data.head()

### Setup Questionnaire Score Dict

In [ ]:
quest_dict = {
    "ads_l": quest_data.filter(like="ADSL").columns,
    "stadi": quest_data.filter(like="STADI_T").columns,
    "stadi-tsst_pre": quest_data.filter(regex="STADI_S_\d{1,2}_TSST_Pre").columns,
    "stadi-tsst_post": quest_data.filter(regex="STADI_S_\d{1,2}_TSST_Post").columns,
    "stadi-ftsst_pre": quest_data.filter(regex="STADI_S_\d{1,2}_fTSST_Pre").columns,
    "stadi-ftsst_post": quest_data.filter(regex="STADI_S_\d{1,2}_fTSST_Post").columns,
    "panas-tsst_pre": quest_data.filter(regex="PANAS_1_\d{1,2}_TSST_Pre").columns,
    "panas-tsst_post": quest_data.filter(regex="PANAS_1_\d{1,2}_TSST_Post").columns,
    "panas-ftsst_pre": quest_data.filter(regex="PANAS_1_\d{1,2}_fTSST_Pre").columns,
    "panas-ftsst_post": quest_data.filter(regex="PANAS_1_\d{1,2}_fTSST_Post").columns,
    "brief_cope": quest_data.filter(like="Brief_Cope").columns,
    "pss": quest_data.filter(like="PSS").columns,
    "bfi_k": quest_data.filter(like="bfik").columns,
    "rsq": quest_data.filter(like="rsq").columns,
    "besaa": quest_data.filter(like="BES").columns,
    "pasa": quest_data.filter(like="PASA").columns,
}
quest_kwargs = {
    "stadi": {"stadi_type": "trait"},
    "stadi-tsst_pre": {"stadi_type": "state"},
    "stadi-tsst_post": {"stadi_type": "state"},
    "stadi-ftsst_pre": {"stadi_type": "state"},
    "stadi-ftsst_post": {"stadi_type": "state"},
    "panas-tsst_pre": {"language": "german"},
    "panas-tsst_post": {"language": "german"},
    "panas-ftsst_pre": {"language": "german"},
    "panas-ftsst_post": {"language": "german"},
}

### Convert Scales to match `BioPsyKit` Implementation

In [ ]:
quest_data_converted = quest_data.copy()
for key in ["ads_l", "pss", "besaa"]:
    quest_data_converted = bp.questionnaires.utils.convert_scale(quest_data_converted, offset=-1, cols=quest_dict[key])

### Process Questionnaire Data

In [ ]:
quest_scores = bp.questionnaires.utils.compute_scores(quest_data_converted, quest_dict, quest_kwargs)
quest_scores.head()

In [ ]:
quest_data["TSST_first"].astype(bool).value_counts()

In [ ]:
quest_scores = quest_data[["age", "gender", "BMI", "TSST_first"]].join(quest_scores)
quest_scores["TSST_first"] = quest_scores["TSST_first"].astype(bool)
quest_scores.head()

In [ ]:
quest_scores.to_csv(output_path.joinpath("stresspose_questionnaire_processed.csv"))